In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.optimize import curve_fit, OptimizeWarning
from tqdm import tqdm
import warnings
from scipy.stats import zscore
from statsmodels.tsa.stattools import acf, pacf
from scipy.optimize import minimize
from vqr import VectorQuantileRegressor
from vqr.solvers.regularized_lse import RegularizedDualVQRSolver
import statsmodels.api as sm


sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dtype_dict = {
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'AnimalNumber': 'Int64',          
    'StartDate': 'str',
    'StartTime': 'str',
    'DateTime': 'str',
    'LactationNumber': 'Int64',       
    'DaysInMilk': 'Int64', 
    'YearSeason': 'str',           
    'TotalYield': 'float',
    'DateTime': 'str',
    'BreedName': 'str',
    'Age': 'Int64',
    'Mother': 'str',
    'Father': 'str',
    'CullDecisionDate': 'str',
    'Temperature': 'float',
    'RelativeHumidity': 'float',      
    'THI_adj': 'float',
    'HW': 'Int64',                    
    'cum_HW': 'Int64',                
    'Temp15Threshold': 'Int64'        
}


# Load the CSV with specified dtypes
data = pd.read_csv('../Data/MergedData/CleanedYieldData.csv', dtype=dtype_dict)

# Convert date and time columns back to datetime and time objects
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data['StartTime'] = pd.to_datetime(data['StartTime'], format='%H:%M:%S', errors='coerce').dt.time
data['StartDate'] = pd.to_datetime(data['StartDate'], errors='coerce')
data['CullDecisionDate'] = pd.to_datetime(data['CullDecisionDate'], errors='coerce')
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data.head()

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,Mother,Father,CullDecisionDate,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096


In [4]:
# Calculate the DailyYield for each cow each day
data['DailyYield'] = data.groupby(['SE_Number', 'StartDate'])['TotalYield'].transform('sum')

# Sort the data by AnimalNumber and StartDate
data.sort_values(['AnimalNumber', 'StartDate'], inplace=True)

# Calculate the previous day's total yield for each cow
data['PreviousDailyYield'] = data.groupby('AnimalNumber')['DailyYield'].shift(1)

# Calculate the daily yield change for each cow
data['DailyYieldChange'] = data['DailyYield'] - data['PreviousDailyYield']

# Group and aggregate data
data = data.groupby(['SE_Number', 'FarmName_Pseudo', 'StartDate']).agg({
    'DailyYield': 'first',
    'PreviousDailyYield': 'first',
    'DailyYieldChange': 'first',
    'HW': 'max',
    'Temperature': 'mean',
    'THI_adj': 'mean',
    'DaysInMilk': 'first',
    'YearSeason': 'first',
    'cum_HW': 'max',
    'Temp15Threshold': 'max',
    'Age': 'first',
    'BreedName': 'first',
    'LactationNumber': 'first'
}).reset_index()

# Renaming and formatting
data.rename(columns={
    'Temperature': 'MeanTemperature',
    'THI_adj': 'MeanTHI_adj',
    'StartDate': 'Date'
}, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])

# Display the first few rows of the transformed data
data.head()

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,BreedName,LactationNumber
0,SE-064c0cec-1189,a624fb9a,2022-01-01,30.77,30.77,0.00,0,-3.025000,28.012944,191,2022-1,0,0,3095,02 SLB,7
1,SE-064c0cec-1189,a624fb9a,2022-01-02,48.22,30.77,17.45,0,-0.279167,32.898193,192,2022-1,0,0,3096,02 SLB,7
2,SE-064c0cec-1189,a624fb9a,2022-01-03,30.53,48.22,-17.69,0,2.033333,36.760487,193,2022-1,0,0,3097,02 SLB,7
3,SE-064c0cec-1189,a624fb9a,2022-01-04,42.26,30.53,11.73,0,0.066667,31.939524,194,2022-1,0,0,3098,02 SLB,7
4,SE-064c0cec-1189,a624fb9a,2022-01-05,38.49,42.26,-3.77,0,-3.700000,26.498206,195,2022-1,0,0,3099,02 SLB,7


In [5]:
# Check if DailyYield is centered around approx the same for each farm
print("Mean of DailyYield:", data.groupby('FarmName_Pseudo')['DailyYield'].mean())
print("Standard Deviation of DailyYield:", data.groupby('FarmName_Pseudo')['DailyYield'].std())

Mean of DailyYield: FarmName_Pseudo
5c06d92d    37.322718
752efd72    31.412607
a624fb9a    34.164215
f454e660    30.811276
Name: DailyYield, dtype: float64
Standard Deviation of DailyYield: FarmName_Pseudo
5c06d92d     9.854998
752efd72     7.760655
a624fb9a    11.417583
f454e660    11.923900
Name: DailyYield, dtype: float64


In [6]:
# Define the Wilmink Lactation Curve function
def wilmink_lactation_curve(dim, a, b, c, d):
    return a + b * dim + c * np.exp(-d * dim)

# Function to remove outliers
def remove_outliers(group, threshold=3.5):
    mean = np.mean(group['DailyYield'])
    std_dev = np.std(group['DailyYield'])
    return group[(group['DailyYield'] > mean - threshold * std_dev) & (group['DailyYield'] < mean + threshold * std_dev)]

# Function to smooth the data using .loc to avoid SettingWithCopyWarning
def smooth_data(group, window=5):
    group.loc[:, 'DailyYield'] = group['DailyYield'].rolling(window, min_periods=1).mean()
    return group

# Function to fit curve_fit before applying Quantile Regression
def fit_with_curve_fit_before_quantreg(dataset, quantile=0.7, max_iter=100000):
    params_dict = {}
    valid_indices = []

    for (animal_number, lactation_number), group in tqdm(dataset.groupby(['SE_Number', 'LactationNumber']), unit=" Segments"):
        try:
            group = remove_outliers(group)
            group = smooth_data(group)
            x_data = group['DaysInMilk'].values.astype(float)
            y_data = group['DailyYield'].values.astype(float)

            # Ensure there are enough data points to fit the curve
            if (len(x_data) < 150) or (len(y_data) < 150):
                print(f"Insufficient data points for cow {animal_number}, lactation {lactation_number}, skipping.")
                continue

            valid_indices.extend(group.index)

            # Fit the model using curve_fit
            try:
                # Initial parameter guesses
                initial_guesses = [np.mean(y_data), 0, np.mean(y_data) / 2, 0.1]
                # Bounds on the parameters to prevent overflow
                bounds = ([-np.inf, -np.inf, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf])

                with warnings.catch_warnings():
                    warnings.filterwarnings('error', category=OptimizeWarning)
                    popt, _ = curve_fit(
                        wilmink_lactation_curve, x_data, y_data,
                        p0=initial_guesses, bounds=bounds, maxfev=30000
                    )

                # Store the parameters in the dictionary
                params_dict[(animal_number, lactation_number)] = {'a': popt[0], 'b': popt[1], 'c': popt[2], 'd': popt[3]}

            except Exception as e:
                print(f"Curve fitting failed for cow {animal_number}, lactation {lactation_number}: {e}")
                continue

            # Now use the parameters from curve_fit for quantile regression
            X = np.column_stack([np.ones_like(x_data), x_data, np.exp(-x_data), -x_data * np.exp(-x_data)])
            quantreg_model = sm.QuantReg(y_data, X)
            quantreg_fit = quantreg_model.fit(q=quantile, max_iter=max_iter, start_params=popt)

            # Update parameters after quantile regression
            a, b, c, d = quantreg_fit.params
            dataset.loc[group.index, 'ExpectedYield'] = wilmink_lactation_curve(group['DaysInMilk'], a, b, c, d)
            params_dict[(animal_number, lactation_number)] = {'a': a, 'b': b, 'c': c, 'd': d}

        except Exception as e:
            print(f"Error processing cow {animal_number}, lactation {lactation_number}: {e}")

    return dataset, params_dict

# Apply the curve fitting before quantile regression
data, params_dict = fit_with_curve_fit_before_quantreg(data, quantile=0.7, max_iter=100000)

# Remove rows where ExpectedYield is NaN
data = data.dropna(subset=['ExpectedYield'])

# Calculate NormalizedDailyYield, PreviousDailyYield, DailyYieldChange, and NormalizedDailyYieldChange
data.loc[:, 'NormalizedDailyYield'] = data['DailyYield'] / data['ExpectedYield']
data.loc[:, 'PreviousDailyYield'] = data.groupby('SE_Number')['DailyYield'].shift(1)
data.loc[:, 'DailyYieldChange'] = data['DailyYield'] - data['PreviousDailyYield']
data.loc[:, 'NormalizedDailyYieldChange'] = data['DailyYieldChange'] / data['ExpectedYield']
data

  0%|          | 4/2746 [00:00<01:11, 38.23 Segments/s]

Insufficient data points for cow SE-064c0cec-1189, lactation 7, skipping.
Insufficient data points for cow SE-30dc5787-1389, lactation 5, skipping.


  0%|          | 10/2746 [00:00<04:12, 10.85 Segments/s]

Insufficient data points for cow SE-5c06d92d-2055, lactation 8, skipping.


  1%|          | 33/2746 [00:01<01:28, 30.56 Segments/s]

...

 99%|█████████▉| 2727/2746 [02:26<00:01, 14.05 Segments/s]/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/pandas/core/arrays/masked.py:672: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs2, **kwargs)
100%|█████████▉| 2740/2746 [02:26<00:00, 21.69 Segments/s]

Insufficient data points for cow SE-f454e660-729, lactation 1, skipping.
Insufficient data points for cow SE-f454e660-735, lactation 2, skipping.


/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (100000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
100%|██████████| 2746/2746 [02:30<00:00, 18.26 Segments/s]

Insufficient data points for cow SE-fcdf259d-0044-0, lactation 9, skipping.


,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,BreedName,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange
88,SE-064c0cec-1189,a624fb9a,2022-05-28,15.22,NaN,NaN,0,9.912500,50.478673,3,2022-2,0,0,3242,02 SLB,8,41.792667,0.364179,<NA>
89,SE-064c0cec-1189,a624fb9a,2022-05-29,18.96,15.22,3.74,0,10.066667,53.841648,4,2022-2,0,0,3243,02 SLB,8,41.712445,0.454541,0.089661
90,SE-064c0cec-1189,a624fb9a,2022-05-30,22.64,18.96,3.68,0,10.466667,52.935959,5,2022-2,0,1,3244,02 SLB,8,41.632223,0.54381,0.088393
91,SE-064c0cec-1189,a624fb9a,2022-05-31,26.49,22.64,3.85,0,11.183333,52.872112,6,2022-2,0,0,3245,02 SLB,8,41.552001,0.637514,0.092655
92,SE-064c0cec-1189,a624fb9a,2022-06-01,33.61,26.49,7.12,0,12.704167,56.056547,7,2022-3,0,1,3246,02 SLB,8,41.471778,0.810431,0.171683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659907,SE-fcdf259d-0044-0,f454e660,2023-06-08,15.14,8.97,6.17,0,15.570833,59.383267,352,2023-3,0,1,4155,41 Fjällko,10,14.629583,1.034889,0.421748
659908,SE-fcdf259d-0044-0,f454e660,2023-06-09,7.47,15.14,-7.67,0,13.254167,54.534255,353,2023-3,0,1,4156,41 Fjällko,10,14.54211,0.513681,-0.527434
659909,SE-fcdf259d-0044-0,f454e660,2023-06-10,14.73,7.47,7.26,0,13.258333,54.082367,354,2023-3,0,1,4157,41 Fjällko,10,14.454637,1.01905,0.502261
659910,SE-fcdf259d-0044-0,f454e660,2023-06-12,12.27,14.73,-2.46,0,15.820833,62.015093,356,2023-3,0,1,4159,41 Fjällko,10,14.279692,0.859262,-0.172273


In [7]:
# Check if NormalizedDailyYield is centered around 1 for each unique farm
print("Mean of NormalizedDailyYield:", data.groupby('FarmName_Pseudo')['NormalizedDailyYield'].mean())
print("Standard Deviation of NormalizedDailyYield:", data.groupby('FarmName_Pseudo')['NormalizedDailyYield'].std())

Mean of NormalizedDailyYield: FarmName_Pseudo
5c06d92d     0.94483
752efd72    0.951713
a624fb9a    0.907158
f454e660    0.912254
Name: NormalizedDailyYield, dtype: Float64
Standard Deviation of NormalizedDailyYield: FarmName_Pseudo
5c06d92d    0.141058
752efd72    0.117541
a624fb9a    0.216816
f454e660    0.247192
Name: NormalizedDailyYield, dtype: Float64


In [8]:
# Define the THI threshold
THI_THRESHOLD = 61

# Calculate the daily heat load based on the THI threshold
data['HeatLoad'] = data['MeanTHI_adj'].apply(lambda x: x - THI_THRESHOLD if x > THI_THRESHOLD else -(THI_THRESHOLD - x))

# Initialize the cumulative heat load column with float type
data['CumulativeHeatLoad'] = 0.0  # Explicitly set as float

data = data.reset_index(drop=True)

# Iterate through the data to calculate cumulative heat load correctly
for i in range(1, len(data)):
    previous_cumulative = data.at[i-1, 'CumulativeHeatLoad']
    current_heat_load = data.at[i, 'HeatLoad']
    
    if current_heat_load < 0:  # If current heat load is negative
        new_cumulative = previous_cumulative + 2 * current_heat_load
    else:
        new_cumulative = previous_cumulative + current_heat_load
    
    # Ensure the cumulative heat load never goes below zero
    if new_cumulative > 0:
        data.at[i, 'CumulativeHeatLoad'] = new_cumulative
    else:
        data.at[i, 'CumulativeHeatLoad'] = 0.0  # Ensure float is maintained

# Drop rows where the 'DailyYield' column has NaN values
data = data.dropna(subset=['DailyYield'])

data.head(-5)

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,...,cum_HW,Temp15Threshold,Age,BreedName,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatLoad,CumulativeHeatLoad
0,SE-064c0cec-1189,a624fb9a,2022-05-28,15.22,NaN,NaN,0,9.912500,50.478673,3,...,0,0,3242,02 SLB,8,41.792667,0.364179,<NA>,-10.521327,0.000000
1,SE-064c0cec-1189,a624fb9a,2022-05-29,18.96,15.22,3.74,0,10.066667,53.841648,4,...,0,0,3243,02 SLB,8,41.712445,0.454541,0.089661,-7.158352,0.000000
2,SE-064c0cec-1189,a624fb9a,2022-05-30,22.64,18.96,3.68,0,10.466667,52.935959,5,...,0,1,3244,02 SLB,8,41.632223,0.54381,0.088393,-8.064041,0.000000
3,SE-064c0cec-1189,a624fb9a,2022-05-31,26.49,22.64,3.85,0,11.183333,52.872112,6,...,0,0,3245,02 SLB,8,41.552001,0.637514,0.092655,-8.127888,0.000000
4,SE-064c0cec-1189,a624fb9a,2022-06-01,33.61,26.49,7.12,0,12.704167,56.056547,7,...,0,1,3246,02 SLB,8,41.471778,0.810431,0.171683,-4.943453,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605438,SE-fcdf259d-0044-0,f454e660,2023-06-03,12.67,15.75,-3.08,0,12.666667,53.132530,347,...,0,1,4150,41 Fjällko,10,15.066946,0.840914,-0.204421,-7.867470,0.000000
605439,SE-fcdf259d-0044-0,f454e660,2023-06-04,22.31,12.67,9.64,0,13.079167,56.726870,348,...,0,1,4151,41 Fjällko,10,14.979474,1.489371,0.643547,-4.273130,0.000000
605440,SE-fcdf259d-0044-0,f454e660,2023-06-05,12.84,22.31,-9.47,0,14.237500,58.482418,349,...,0,1,4152,41 Fjällko,10,14.892001,0.862208,-0.635912,-2.517582,0.000000
605441,SE-fcdf259d-0044-0,f454e660,2023-06-06,9.47,12.84,-3.37,0,15.345833,60.546358,350,...,0,1,4153,41 Fjällko,10,14.804528,0.639669,-0.227633,-0.453642,0.000000


In [9]:
# When CumulativeHeatLoad is greater than 3, it indicates that the cow is under heat stress
data['HeatStress'] = (data['CumulativeHeatLoad'] > 3).astype(int)
data.head(-5)

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,...,Temp15Threshold,Age,BreedName,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatLoad,CumulativeHeatLoad,HeatStress
0,SE-064c0cec-1189,a624fb9a,2022-05-28,15.22,NaN,NaN,0,9.912500,50.478673,3,...,0,3242,02 SLB,8,41.792667,0.364179,<NA>,-10.521327,0.000000,0
1,SE-064c0cec-1189,a624fb9a,2022-05-29,18.96,15.22,3.74,0,10.066667,53.841648,4,...,0,3243,02 SLB,8,41.712445,0.454541,0.089661,-7.158352,0.000000,0
2,SE-064c0cec-1189,a624fb9a,2022-05-30,22.64,18.96,3.68,0,10.466667,52.935959,5,...,1,3244,02 SLB,8,41.632223,0.54381,0.088393,-8.064041,0.000000,0
3,SE-064c0cec-1189,a624fb9a,2022-05-31,26.49,22.64,3.85,0,11.183333,52.872112,6,...,0,3245,02 SLB,8,41.552001,0.637514,0.092655,-8.127888,0.000000,0
4,SE-064c0cec-1189,a624fb9a,2022-06-01,33.61,26.49,7.12,0,12.704167,56.056547,7,...,1,3246,02 SLB,8,41.471778,0.810431,0.171683,-4.943453,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605438,SE-fcdf259d-0044-0,f454e660,2023-06-03,12.67,15.75,-3.08,0,12.666667,53.132530,347,...,1,4150,41 Fjällko,10,15.066946,0.840914,-0.204421,-7.867470,0.000000,0
605439,SE-fcdf259d-0044-0,f454e660,2023-06-04,22.31,12.67,9.64,0,13.079167,56.726870,348,...,1,4151,41 Fjällko,10,14.979474,1.489371,0.643547,-4.273130,0.000000,0
605440,SE-fcdf259d-0044-0,f454e660,2023-06-05,12.84,22.31,-9.47,0,14.237500,58.482418,349,...,1,4152,41 Fjällko,10,14.892001,0.862208,-0.635912,-2.517582,0.000000,0
605441,SE-fcdf259d-0044-0,f454e660,2023-06-06,9.47,12.84,-3.37,0,15.345833,60.546358,350,...,1,4153,41 Fjällko,10,14.804528,0.639669,-0.227633,-0.453642,0.000000,0


In [10]:
# Make a dataframe from the parameters dictionary, it should contain Se_Number, LactationNumber, a, b, c, d
params_df = pd.DataFrame(params_dict).T.reset_index()
params_df.columns = ['SE_Number', 'LactationNumber', 'a', 'b', 'c', 'd']
params_df.head(-5)

,SE_Number,LactationNumber,a,b,c,d
0,SE-064c0cec-1189,8,42.033334,-0.080222,1.898115e+03,8.106137e+02
1,SE-30dc5787-1389,6,53.527867,-0.078718,2.219759e+03,8.703531e+02
2,SE-30dc5787-1389,7,50.409783,-0.102973,2.765920e+00,1.116788e+02
3,SE-30dc5787-1396,5,35.152650,-0.024675,-6.560076e-92,1.212551e-89
4,SE-30dc5787-1396,6,56.606093,-0.078237,1.683317e+00,1.470512e+02
...,...,...,...,...,...,...
2029,SE-f454e660-778,1,23.615367,0.029837,7.821117e+02,3.860034e+02
2030,SE-f454e660-780,1,20.483449,0.003832,2.093686e+03,7.422069e+02
2031,SE-f454e660-787,1,35.441859,0.001323,1.359524e+03,5.631486e+02
2032,SE-f454e660-788,1,20.860446,0.017855,4.799397e+03,1.477833e+03


In [11]:
# Calculate Z-scores for each parameter
params_df['z_a'] = zscore(params_df['a'])
params_df['z_b'] = zscore(params_df['b'])
params_df['z_c'] = zscore(params_df['c'])
params_df['z_d'] = zscore(params_df['d'])

params_df.head(-5)

,SE_Number,LactationNumber,a,b,c,d,z_a,z_b,z_c,z_d
0,SE-064c0cec-1189,8,42.033334,-0.080222,1.898115e+03,8.106137e+02,-0.243702,-0.924142,-0.033840,-0.062644
1,SE-30dc5787-1389,6,53.527867,-0.078718,2.219759e+03,8.703531e+02,0.842135,-0.879756,0.025373,-0.050381
2,SE-30dc5787-1389,7,50.409783,-0.102973,2.765920e+00,1.116788e+02,0.547584,-1.595556,-0.382764,-0.206116
3,SE-30dc5787-1396,5,35.152650,-0.024675,-6.560076e-92,1.212551e-89,-0.893690,0.715146,-0.383273,-0.229041
4,SE-30dc5787-1396,6,56.606093,-0.078237,1.683317e+00,1.470512e+02,1.132922,-0.865558,-0.382963,-0.198855
...,...,...,...,...,...,...,...,...,...,...
2029,SE-f454e660-778,1,23.615367,0.029837,7.821117e+02,3.860034e+02,-1.983566,2.323912,-0.239290,-0.149805
2030,SE-f454e660-780,1,20.483449,0.003832,2.093686e+03,7.422069e+02,-2.279424,1.556437,0.002164,-0.076686
2031,SE-f454e660-787,1,35.441859,0.001323,1.359524e+03,5.631486e+02,-0.866370,1.482398,-0.132992,-0.113442
2032,SE-f454e660-788,1,20.860446,0.017855,4.799397e+03,1.477833e+03,-2.243811,1.970282,0.500271,0.074319


In [12]:
# Identify outliers (using Z-score > 3.5 or < -3.5 as threshold)
outliers = params_df[(np.abs(params_df[['z_a', 'z_b', 'z_c', 'z_d']]) > 3.5).any(axis=1)]

x = outliers.count()
print("Number of outliers:", x)

# Optionally, drop the outliers
params_df_cleaned = params_df.drop(outliers.index)
params_df_cleaned.head(-5)

Number of outliers: SE_Number          51
LactationNumber    51
a                  51
b                  51
c                  51
d                  51
z_a                51
z_b                51
z_c                51
z_d                51
dtype: int64


,SE_Number,LactationNumber,a,b,c,d,z_a,z_b,z_c,z_d
0,SE-064c0cec-1189,8,42.033334,-0.080222,1.898115e+03,8.106137e+02,-0.243702,-0.924142,-0.033840,-0.062644
1,SE-30dc5787-1389,6,53.527867,-0.078718,2.219759e+03,8.703531e+02,0.842135,-0.879756,0.025373,-0.050381
2,SE-30dc5787-1389,7,50.409783,-0.102973,2.765920e+00,1.116788e+02,0.547584,-1.595556,-0.382764,-0.206116
3,SE-30dc5787-1396,5,35.152650,-0.024675,-6.560076e-92,1.212551e-89,-0.893690,0.715146,-0.383273,-0.229041
4,SE-30dc5787-1396,6,56.606093,-0.078237,1.683317e+00,1.470512e+02,1.132922,-0.865558,-0.382963,-0.198855
...,...,...,...,...,...,...,...,...,...,...
2027,SE-f454e660-750,1,32.711090,-0.001080,6.658862e+03,2.062282e+03,-1.124333,1.411480,0.842589,0.194291
2028,SE-f454e660-767,1,26.261397,0.045251,5.227893e+03,1.437320e+03,-1.733607,2.778795,0.579155,0.066003
2029,SE-f454e660-778,1,23.615367,0.029837,7.821117e+02,3.860034e+02,-1.983566,2.323912,-0.239290,-0.149805
2030,SE-f454e660-780,1,20.483449,0.003832,2.093686e+03,7.422069e+02,-2.279424,1.556437,0.002164,-0.076686


In [13]:
# Identify unique SE_Number and LactationNumber combinations from the outliers
outlier_combinations = outliers[['SE_Number', 'LactationNumber']].drop_duplicates()

# Merge with the original data to find rows that match these outlier combinations
data_cleaned = data.merge(outlier_combinations, on=['SE_Number', 'LactationNumber'], how='left', indicator=True)

# Keep only the rows that do not match the outlier combinations
data_cleaned = data_cleaned[data_cleaned['_merge'] == 'left_only'].drop(columns=['_merge'])

# Now data_cleaned contains the original data with the outlier combinations removed
print("Number of rows removed:", len(data) - len(data_cleaned))
data_cleaned.head(-5)

Number of rows removed: 15389


,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,...,Temp15Threshold,Age,BreedName,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatLoad,CumulativeHeatLoad,HeatStress
0,SE-064c0cec-1189,a624fb9a,2022-05-28,15.22,NaN,NaN,0,9.912500,50.478673,3,...,0,3242,02 SLB,8,41.792667,0.364179,<NA>,-10.521327,0.000000,0
1,SE-064c0cec-1189,a624fb9a,2022-05-29,18.96,15.22,3.74,0,10.066667,53.841648,4,...,0,3243,02 SLB,8,41.712445,0.454541,0.089661,-7.158352,0.000000,0
2,SE-064c0cec-1189,a624fb9a,2022-05-30,22.64,18.96,3.68,0,10.466667,52.935959,5,...,1,3244,02 SLB,8,41.632223,0.54381,0.088393,-8.064041,0.000000,0
3,SE-064c0cec-1189,a624fb9a,2022-05-31,26.49,22.64,3.85,0,11.183333,52.872112,6,...,0,3245,02 SLB,8,41.552001,0.637514,0.092655,-8.127888,0.000000,0
4,SE-064c0cec-1189,a624fb9a,2022-06-01,33.61,26.49,7.12,0,12.704167,56.056547,7,...,1,3246,02 SLB,8,41.471778,0.810431,0.171683,-4.943453,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605438,SE-fcdf259d-0044-0,f454e660,2023-06-03,12.67,15.75,-3.08,0,12.666667,53.132530,347,...,1,4150,41 Fjällko,10,15.066946,0.840914,-0.204421,-7.867470,0.000000,0
605439,SE-fcdf259d-0044-0,f454e660,2023-06-04,22.31,12.67,9.64,0,13.079167,56.726870,348,...,1,4151,41 Fjällko,10,14.979474,1.489371,0.643547,-4.273130,0.000000,0
605440,SE-fcdf259d-0044-0,f454e660,2023-06-05,12.84,22.31,-9.47,0,14.237500,58.482418,349,...,1,4152,41 Fjällko,10,14.892001,0.862208,-0.635912,-2.517582,0.000000,0
605441,SE-fcdf259d-0044-0,f454e660,2023-06-06,9.47,12.84,-3.37,0,15.345833,60.546358,350,...,1,4153,41 Fjällko,10,14.804528,0.639669,-0.227633,-0.453642,0.000000,0


In [14]:
# Check if NormalizedDailyYield is centered around 1 for each unique farm
print("Mean of NormalizedDailyYield:", data_cleaned.groupby('FarmName_Pseudo')['NormalizedDailyYield'].mean())
print("Standard Deviation of NormalizedDailyYield:", data_cleaned.groupby('FarmName_Pseudo')['NormalizedDailyYield'].std())

Mean of NormalizedDailyYield: FarmName_Pseudo
5c06d92d    0.945267
752efd72    0.951719
a624fb9a    0.907128
f454e660    0.912273
Name: NormalizedDailyYield, dtype: Float64
Standard Deviation of NormalizedDailyYield: FarmName_Pseudo
5c06d92d    0.140407
752efd72    0.117494
a624fb9a    0.216923
f454e660    0.248424
Name: NormalizedDailyYield, dtype: Float64


In [15]:
data_cleaned['Residuals'] = data_cleaned['DailyYield'] - data_cleaned['ExpectedYield']
data_cleaned

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,...,Age,BreedName,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatLoad,CumulativeHeatLoad,HeatStress,Residuals
0,SE-064c0cec-1189,a624fb9a,2022-05-28,15.22,NaN,NaN,0,9.912500,50.478673,3,...,3242,02 SLB,8,41.792667,0.364179,<NA>,-10.521327,0.000000,0,-26.572667
1,SE-064c0cec-1189,a624fb9a,2022-05-29,18.96,15.22,3.74,0,10.066667,53.841648,4,...,3243,02 SLB,8,41.712445,0.454541,0.089661,-7.158352,0.000000,0,-22.752445
2,SE-064c0cec-1189,a624fb9a,2022-05-30,22.64,18.96,3.68,0,10.466667,52.935959,5,...,3244,02 SLB,8,41.632223,0.54381,0.088393,-8.064041,0.000000,0,-18.992223
3,SE-064c0cec-1189,a624fb9a,2022-05-31,26.49,22.64,3.85,0,11.183333,52.872112,6,...,3245,02 SLB,8,41.552001,0.637514,0.092655,-8.127888,0.000000,0,-15.062001
4,SE-064c0cec-1189,a624fb9a,2022-06-01,33.61,26.49,7.12,0,12.704167,56.056547,7,...,3246,02 SLB,8,41.471778,0.810431,0.171683,-4.943453,0.000000,0,-7.861778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605443,SE-fcdf259d-0044-0,f454e660,2023-06-08,15.14,8.97,6.17,0,15.570833,59.383267,352,...,4155,41 Fjällko,10,14.629583,1.034889,0.421748,-1.616733,0.000000,0,0.510417
605444,SE-fcdf259d-0044-0,f454e660,2023-06-09,7.47,15.14,-7.67,0,13.254167,54.534255,353,...,4156,41 Fjällko,10,14.54211,0.513681,-0.527434,-6.465745,0.000000,0,-7.07211
605445,SE-fcdf259d-0044-0,f454e660,2023-06-10,14.73,7.47,7.26,0,13.258333,54.082367,354,...,4157,41 Fjällko,10,14.454637,1.01905,0.502261,-6.917633,0.000000,0,0.275363
605446,SE-fcdf259d-0044-0,f454e660,2023-06-12,12.27,14.73,-2.46,0,15.820833,62.015093,356,...,4159,41 Fjällko,10,14.279692,0.859262,-0.172273,1.015093,1.015093,0,-2.009692


In [16]:
# Group by 'FarmName_Pseudo' to perform calculations at the farm level
farm_results = []

for farm_name, farm_group in data_cleaned.groupby('FarmName_Pseudo'):
    farm_residuals = []
    
    for se_number, cow_group in farm_group.groupby('SE_Number'):
        residuals = cow_group['Residuals']
        
        if len(residuals) > 1:  # Ensure there are residuals to analyze
            farm_residuals.append(residuals)
    
    if len(farm_residuals) > 0:
        # Combine residuals from all cows in the farm
        combined_residuals = np.concatenate(farm_residuals)
        
        # Calculate farm-level statistics
        mean_residuals = combined_residuals.mean()
        std_residuals = combined_residuals.std()
        acf_values = acf(combined_residuals, nlags=30, fft=False)
        pacf_values = pacf(combined_residuals, nlags=min(30, len(combined_residuals)//2))

        # Print the farm-level statistics
        print(f"Farm: {farm_name}")
        print(f"Mean Residuals: {mean_residuals}")
        print(f"Standard Deviation of Residuals: {std_residuals}")
        print(f"ACF (first 5 lags): {acf_values[:5]}")
        print(f"PACF (first 5 lags): {pacf_values[:5]}")
        print("=" * 50)


Farm: 5c06d92d
Mean Residuals: -2.2208057468633102
Standard Deviation of Residuals: 5.901558036684177
ACF (first 5 lags): [1.         0.45033682 0.42398472 0.37190396 0.33789599]
PACF (first 5 lags): [1.         0.4503387  0.27745195 0.14813144 0.09485012]
Farm: 752efd72
Mean Residuals: -1.6140158008827528
Standard Deviation of Residuals: 4.0104053105833755
ACF (first 5 lags): [1.         0.47760427 0.44521024 0.40059334 0.35844906]
PACF (first 5 lags): [1.         0.47760678 0.28126571 0.15958209 0.08931462]
Farm: a624fb9a
Mean Residuals: -3.6913673226079333
Standard Deviation of Residuals: 8.625193330567392
ACF (first 5 lags): [1.         0.50335434 0.63869248 0.6122737  0.5971928 ]
PACF (first 5 lags): [1.         0.50336104 0.51610349 0.35953799 0.22438246]


/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/numpy/_core/_methods.py:127: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/numpy/_core/_methods.py:164: RuntimeWarning: invalid value encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


Farm: f454e660
Mean Residuals: nan
Standard Deviation of Residuals: nan
ACF (first 5 lags): [nan nan nan nan nan]
PACF (first 5 lags): [ 1. nan nan nan nan]


/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/numpy/_core/_methods.py:127: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


In [17]:
# Group by 'FarmName_Pseudo', 'SE_Number', and 'LactationNumber' to perform individual calculations
farm_results = []

for farm_name, farm_group in data_cleaned.groupby('FarmName_Pseudo'):
    print(f"Farm: {farm_name}")
    
    for (se_number, lactation_number), cow_group in farm_group.groupby(['SE_Number', 'LactationNumber']):
        residuals = cow_group['Residuals']
        
        if len(residuals) > 1:  # Ensure there are residuals to analyze
            mean_residuals = residuals.mean()
            std_residuals = residuals.std()
            acf_values = acf(residuals, nlags=30, fft=False)
            pacf_values = pacf(residuals, nlags=min(30, len(residuals)//2))

            # Print the statistics
            print(f"\nCow: {se_number}, Lactation Number: {lactation_number}")
            print(f"Mean Residuals: {mean_residuals}")
            print(f"Standard Deviation of Residuals: {std_residuals}")
            print(f"ACF (first 5 lags): {acf_values[:5]}")
            print(f"PACF (first 5 lags): {pacf_values[:5]}")
            print("-" * 50)
            
    print("=" * 50)

Farm: 5c06d92d

Cow: SE-5c06d92d-2000, Lactation Number: 8
Mean Residuals: -1.428445147952014
Standard Deviation of Residuals: 3.340228771167533
ACF (first 5 lags): [1.         0.18872437 0.2788504  0.16545083 0.16538944]
PACF (first 5 lags): [1.         0.18943923 0.25420648 0.08814745 0.07086122]
--------------------------------------------------

Cow: SE-5c06d92d-2016, Lactation Number: 8
Mean Residuals: -2.8510760133444593
Standard Deviation of Residuals: 7.541318601957895
ACF (first 5 lags): [1.         0.26713025 0.25739877 0.27546499 0.28108091]
PACF (first 5 lags): [1.         0.26804823 0.20182519 0.18956226 0.16715481]
--------------------------------------------------

Cow: SE-5c06d92d-2055, Lactation Number: 6
Mean Residuals: -1.7910419628600156
Standard Deviation of Residuals: 4.80408040639851
ACF (first 5 lags): [ 1.          0.08095262  0.32228993  0.02447001 -0.00513046]
PACF (first 5 lags): [ 1.          0.08142056  0.32153976 -0.02165918 -0.12385351]
-----------------

/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:472: RuntimeWarning: invalid value encountered in subtract
  xo = x - x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1473: RuntimeWarning: invalid value encountered in subtract
  x -= x.mean()



Cow: SE-f454e660-514, Lactation Number: 3
Mean Residuals: -1.672514331566524
Standard Deviation of Residuals: 5.682984054852484
ACF (first 5 lags): [ 1.         -0.01568314  0.17424634  0.13596907  0.20287893]
PACF (first 5 lags): [ 1.         -0.01572954  0.17507635  0.14674449  0.1918216 ]
--------------------------------------------------

Cow: SE-f454e660-516, Lactation Number: 2
Mean Residuals: -1.74670401441688
Standard Deviation of Residuals: 6.077283432575313
ACF (first 5 lags): [ 1.         -0.10443502  0.20723386  0.07503106  0.20159771]
PACF (first 5 lags): [ 1.         -0.10478904  0.19985746  0.12031     0.19378763]
--------------------------------------------------

Cow: SE-f454e660-516, Lactation Number: 3
Mean Residuals: -3.83377164589266
Standard Deviation of Residuals: 8.673273913075134
ACF (first 5 lags): [1.         0.06941399 0.34649081 0.28835563 0.34230895]
PACF (first 5 lags): [1.         0.06956359 0.34481704 0.28555026 0.27975844]
----------------------------

/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:472: RuntimeWarning: invalid value encountered in subtract
  xo = x - x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1473: RuntimeWarning: invalid value encountered in subtract
  x -= x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:472: RuntimeWarning: invalid value encountered in subtract
  xo = x - x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1473: RuntimeWarning: invalid value encountered in subtract
  x -= x.mean()


In [18]:
# Define the thresholds
mean_residual_threshold = 0.075
std_residual_threshold = 7.5
acf_threshold = 0.25
pacf_threshold = 0.25

# List to collect flagged combinations
flagged_combinations = []

for farm_name, farm_group in data_cleaned.groupby('FarmName_Pseudo'):
    for (se_number, lactation_number), cow_group in farm_group.groupby(['SE_Number', 'LactationNumber']):
        residuals = cow_group['Residuals'].dropna()
        
        if len(residuals) > 1:  # Ensure there are residuals to analyze
            mean_residuals = residuals.mean()
            std_residuals = residuals.std()
            acf_values = acf(residuals, nlags=30, fft=False)
            pacf_values = pacf(residuals, nlags=min(30, len(residuals)//2))

            # Check against thresholds
            if (abs(mean_residuals) > mean_residual_threshold or 
                std_residuals > std_residual_threshold or 
                abs(acf_values[1]) > acf_threshold or 
                abs(pacf_values[1]) > pacf_threshold):
                
                # Collect the combination if it exceeds any threshold
                flagged_combinations.append({
                    'Farm': farm_name,
                    'SE_Number': se_number,
                    'LactationNumber': lactation_number,
                    'Mean Residuals': mean_residuals,
                    'Std Residuals': std_residuals,
                    'ACF[1]': acf_values[1],
                    'PACF[1]': pacf_values[1]
                })

# Convert to a DataFrame for easier inspection
flagged_df = pd.DataFrame(flagged_combinations)
flagged_df

/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:472: RuntimeWarning: invalid value encountered in subtract
  xo = x - x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1473: RuntimeWarning: invalid value encountered in subtract
  x -= x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:472: RuntimeWarning: invalid value encountered in subtract
  xo = x - x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1473: RuntimeWarning: invalid value encountered in subtract
  x -= x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:472: RuntimeWarning: invalid value encountered in subtract
  xo = x - x.mean()
/Users/user/anaconda3/envs/GIGACOW/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1473: RuntimeWarning: i

,Farm,SE_Number,LactationNumber,Mean Residuals,Std Residuals,ACF[1],PACF[1]
0,5c06d92d,SE-5c06d92d-2000,8,-1.428445,3.340229,0.188724,0.189439
1,5c06d92d,SE-5c06d92d-2016,8,-2.851076,7.541319,0.267130,0.268048
2,5c06d92d,SE-5c06d92d-2055,6,-1.791042,4.80408,0.080953,0.081421
3,5c06d92d,SE-5c06d92d-2055,7,-3.734919,9.275382,0.448348,0.449560
4,5c06d92d,SE-5c06d92d-2058,7,-1.772330,4.488173,-0.021354,-0.021454
...,...,...,...,...,...,...,...
1983,f454e660,SE-f454e660-788,1,-1.430812,6.218114,0.057208,0.057374
1984,f454e660,SE-f454e660-799,1,-3.277432,8.406002,0.240954,0.241590
1985,f454e660,SE-f454e660-804,1,-2.272673,7.305238,0.059572,0.059744
1986,f454e660,SE-f454e660-810,1,-3.410274,8.433515,0.193095,0.193574


In [19]:
# Define the Wilmink Lactation Curve function
def wilmink_lactation_curve(dim, a, b, c, d):
    dim = np.array(dim, dtype=float)
    return a + b * dim + c * np.exp(-d * dim)

# Function to directly refit the Wilmink Lactation Curve (Standard Process)
def refit_wilmink(cow_data):
    x_data = cow_data['DaysInMilk'].values
    y_data = cow_data['DailyYield'].values

    # Use initial guesses and bounds from the original fitting process
    initial_guesses = [np.mean(y_data), 0, np.mean(y_data) / 2, 0.1]
    bounds = ([-np.inf, -np.inf, -np.inf, 0], [np.inf, np.inf, np.inf, np.inf])

    popt, _ = curve_fit(wilmink_lactation_curve, x_data, y_data, p0=initial_guesses, bounds=bounds, maxfev=30000)
    
    # Calculate the expected yield with the refitted parameters
    cow_data['ExpectedYield'] = wilmink_lactation_curve(cow_data['DaysInMilk'], *popt)
    
    # Calculate new residuals
    cow_data['Residuals'] = cow_data['DailyYield'] - cow_data['ExpectedYield']
    
    return cow_data

# Function to add lagged variables for addressing autocorrelation
def add_lagged_variables(cow_data, max_lag=3):
    for lag in range(1, max_lag + 1):
        cow_data[f'lag_{lag}'] = cow_data['DailyYield'].shift(lag)
    return cow_data.dropna()

# Define the Robust Wilmink Lactation Curve function
def robust_wilmink_lactation_curve(dim, a, b, c, d, lag1, lag2, lag3):
    dim = np.array(dim, dtype=np.float64)
    days_in_milk = dim[0]
    lag_1 = dim[1]
    lag_2 = dim[2]
    lag_3 = dim[3]
    
    return a + b * days_in_milk + c * np.exp(-d * days_in_milk) + lag1 * lag_1 + lag2 * lag_2 + lag3 * lag_3

# Function to fit the robust Wilmink model
def fit_robust_wilmink(cow_data, lags=3):
    cow_data = add_lagged_variables(cow_data, max_lag=lags)
    
    x_data = cow_data[['DaysInMilk', 'lag_1', 'lag_2', 'lag_3']].values.T
    y_data = cow_data['DailyYield'].values
    
    initial_guesses = [np.mean(y_data), 0, np.mean(y_data) / 2, 0.1, 0, 0, 0]
    bounds = ([-np.inf, -np.inf, -np.inf, 0, -np.inf, -np.inf, -np.inf], 
              [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf])
    
    try:
        popt, _ = curve_fit(robust_wilmink_lactation_curve, x_data, y_data, p0=initial_guesses, bounds=bounds, maxfev=50000)
        cow_data.loc[:, 'ExpectedYield'] = robust_wilmink_lactation_curve(x_data, *popt)
        cow_data.loc[:, 'Residuals'] = cow_data['DailyYield'] - cow_data['ExpectedYield']
    except RuntimeError as e:
        print(f"Curve fitting failed: {e}")
        cow_data.loc[:, 'ExpectedYield'] = np.nan
        cow_data.loc[:, 'Residuals'] = np.nan
    
    return cow_data

# Function to add lagged variables for addressing autocorrelation
def add_lagged_variables(cow_data, max_lag=3):
    for lag in range(1, max_lag + 1):
        cow_data[f'lag_{lag}'] = cow_data['DailyYield'].shift(lag)
    return cow_data.dropna()

# Example usage: Applying the robust model to flagged cases
for index, row in flagged_df.iterrows():
    se_number = row['SE_Number']
    lactation_number = row['LactationNumber']
    
    cow_data = data_cleaned[(data_cleaned['SE_Number'] == se_number) & 
                            (data_cleaned['LactationNumber'] == lactation_number)].copy()
    
    if abs(row['ACF[1]']) > 0.2:  # Significant autocorrelation
        cow_data = add_lagged_variables(cow_data, max_lag=3)
        cow_data_refitted = fit_robust_wilmink(cow_data, lags=3)
        data_cleaned.update(cow_data_refitted)
    else:
        cow_data_refitted = refit_wilmink(cow_data)
        data_cleaned.update(cow_data_refitted)

# Erase all rows where ExpectedYield is NaN
data_cleaned = data_cleaned.dropna(subset=['ExpectedYield']).reset_index(drop=True)

data_cleaned['NormalizedDailyYield'] = data_cleaned['DailyYield'] / data_cleaned['ExpectedYield']
data_cleaned['NormalizedDailyYieldChange'] = data_cleaned['DailyYieldChange'] / data_cleaned['ExpectedYield']

data_cleaned

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,...,Age,BreedName,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatLoad,CumulativeHeatLoad,HeatStress,Residuals
0,SE-064c0cec-1189,a624fb9a,2022-05-28,15.22,NaN,NaN,0,9.912500,50.478673,3,...,3242,02 SLB,8,14.019488,1.085632,<NA>,-10.521327,0.000000,0,1.200512
1,SE-064c0cec-1189,a624fb9a,2022-05-29,18.96,15.22,3.74,0,10.066667,53.841648,4,...,3243,02 SLB,8,20.100106,0.943279,0.186069,-7.158352,0.000000,0,-1.140106
2,SE-064c0cec-1189,a624fb9a,2022-05-30,22.64,18.96,3.68,0,10.466667,52.935959,5,...,3244,02 SLB,8,24.743239,0.914997,0.148727,-8.064041,0.000000,0,-2.103239
3,SE-064c0cec-1189,a624fb9a,2022-05-31,26.49,22.64,3.85,0,11.183333,52.872112,6,...,3245,02 SLB,8,28.284407,0.936558,0.136117,-8.127888,0.000000,0,-1.794407
4,SE-064c0cec-1189,a624fb9a,2022-06-01,33.61,26.49,7.12,0,12.704167,56.056547,7,...,3246,02 SLB,8,30.980818,1.084865,0.22982,-4.943453,0.000000,0,2.629182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590054,SE-fcdf259d-0044-0,f454e660,2023-06-08,15.14,8.97,6.17,0,15.570833,59.383267,352,...,4155,41 Fjällko,10,11.914894,1.270678,0.517839,-1.616733,0.000000,0,3.225106
590055,SE-fcdf259d-0044-0,f454e660,2023-06-09,7.47,15.14,-7.67,0,13.254167,54.534255,353,...,4156,41 Fjällko,10,10.937277,0.682985,-0.701271,-6.465745,0.000000,0,-3.467277
590056,SE-fcdf259d-0044-0,f454e660,2023-06-10,14.73,7.47,7.26,0,13.258333,54.082367,354,...,4157,41 Fjällko,10,12.646376,1.164761,0.574077,-6.917633,0.000000,0,2.083624
590057,SE-fcdf259d-0044-0,f454e660,2023-06-12,12.27,14.73,-2.46,0,15.820833,62.015093,356,...,4159,41 Fjällko,10,11.04863,1.110545,-0.222652,1.015093,1.015093,0,1.22137


In [20]:
# Define the thresholds
mean_residual_threshold = 0.075
std_residual_threshold = 7.5
acf_threshold = 0.25
pacf_threshold = 0.25

# List to collect flagged combinations
flagged_combinations = []

for farm_name, farm_group in data_cleaned.groupby('FarmName_Pseudo'):
    for (se_number, lactation_number), cow_group in farm_group.groupby(['SE_Number', 'LactationNumber']):
        residuals = cow_group['Residuals'].dropna()
        
        if len(residuals) > 1:  # Ensure there are residuals to analyze
            mean_residuals = residuals.mean()
            std_residuals = residuals.std()
            acf_values = acf(residuals, nlags=30, fft=False)
            pacf_values = pacf(residuals, nlags=min(30, len(residuals)//2))

            # Check against thresholds
            if (abs(mean_residuals) > mean_residual_threshold or 
                std_residuals > std_residual_threshold or 
                abs(acf_values[1]) > acf_threshold or 
                abs(pacf_values[1]) > pacf_threshold):
                
                # Collect the combination if it exceeds any threshold
                flagged_combinations.append({
                    'Farm': farm_name,
                    'SE_Number': se_number,
                    'LactationNumber': lactation_number,
                    'Mean Residuals': mean_residuals,
                    'Std Residuals': std_residuals,
                    'ACF[1]': acf_values[1],
                    'PACF[1]': pacf_values[1]
                })

# Convert to a DataFrame for easier inspection
flagged_df = pd.DataFrame(flagged_combinations)
flagged_df

,Farm,SE_Number,LactationNumber,Mean Residuals,Std Residuals,ACF[1],PACF[1]
0,5c06d92d,SE-5c06d92d-2016,8,-1.654669e-01,6.625292,0.013884,0.013932
1,5c06d92d,SE-5c06d92d-2055,7,-3.825457e-01,8.168593,0.061518,0.061684
2,5c06d92d,SE-5c06d92d-2058,8,-5.966886e-01,7.910538,0.247368,0.248096
3,5c06d92d,SE-5c06d92d-2211,7,-1.215006e+00,10.445431,0.456755,0.458870
4,5c06d92d,SE-5c06d92d-2246,6,-1.189456e+00,11.110472,0.300373,0.302109
...,...,...,...,...,...,...,...
1551,f454e660,SE-f454e660-767,1,-9.575293e-02,7.464308,-0.103453,-0.103664
1552,f454e660,SE-f454e660-787,1,-1.978801e-01,8.388303,0.015798,0.015844
1553,f454e660,SE-f454e660-799,1,-2.583064e-01,7.189122,0.017134,0.017179
1554,f454e660,SE-f454e660-810,1,2.824120e-08,8.230547,0.160006,0.160403


In [21]:
def remove_outliers(data, threshold=3.5):
    # Calculate z-scores of residuals
    data = data.copy()  # Create a copy to avoid the SettingWithCopyWarning
    data['z_score'] = (data['Residuals'] - data['Residuals'].mean()) / data['Residuals'].std()
    
    # Identify the number of outliers
    num_outliers = (data['z_score'].abs() >= threshold).sum()
    print(f"Number of outliers detected: {num_outliers}")
    
    # Remove rows where the z-score of the residual is greater than the threshold
    cleaned_data = data.loc[(data['z_score'].abs() < threshold)].drop(columns=['z_score'])
    
    # Print the number of rows before and after
    print(f"Number of rows before outlier removal: {len(data)}")
    print(f"Number of rows after outlier removal: {len(cleaned_data)}")
    
    return cleaned_data

# Apply to flagged cases
for index, row in flagged_df.iterrows():
    se_number = row['SE_Number']
    lactation_number = row['LactationNumber']
    
    # Select the cow data for the specific SE_Number and LactationNumber
    cow_data = data_cleaned.loc[(data_cleaned['SE_Number'] == se_number) & 
                                (data_cleaned['LactationNumber'] == lactation_number)]
    
    # Remove outliers
    cow_data_trimmed = remove_outliers(cow_data, threshold=3.5)
    
    # Recalculate the residuals and update the dataset
    cow_data_trimmed['Residuals'] = cow_data_trimmed['DailyYield'] - cow_data_trimmed['ExpectedYield']
    
    # Remove the old data for this cow from data_cleaned
    data_cleaned = data_cleaned.loc[~((data_cleaned['SE_Number'] == se_number) & 
                                      (data_cleaned['LactationNumber'] == lactation_number))]
    
    # Append the cleaned data back to data_cleaned
    data_cleaned = pd.concat([data_cleaned, cow_data_trimmed], ignore_index=True)

Number of outliers detected: 1
Number of rows before outlier removal: 292
Number of rows after outlier removal: 291
Number of outliers detected: 4
Number of rows before outlier removal: 371
Number of rows after outlier removal: 367
Number of outliers detected: 6
Number of rows before outlier removal: 341
Number of rows after outlier removal: 335
Number of outliers detected: 4
Number of rows before outlier removal: 217
Number of rows after outlier removal: 213
Number of outliers detected: 0
Number of rows before outlier removal: 174
Number of rows after outlier removal: 174
Number of outliers detected: 2
Number of rows before outlier removal: 483
Number of rows after outlier removal: 481
Number of outliers detected: 5
Number of rows before outlier removal: 362
Number of rows after outlier removal: 357
Number of outliers detected: 0
Number of rows before outlier removal: 340
Number of rows after outlier removal: 340
Number of outliers detected: 5
Number of rows before outlier removal: 42

In [22]:
data_cleaned

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,...,Age,BreedName,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatLoad,CumulativeHeatLoad,HeatStress,Residuals
0,SE-064c0cec-1189,a624fb9a,2022-05-28,15.22,NaN,NaN,0,9.912500,50.478673,3,...,3242,02 SLB,8,14.019488,1.085632,<NA>,-10.521327,0.000000,0,1.200512
1,SE-064c0cec-1189,a624fb9a,2022-05-29,18.96,15.22,3.74,0,10.066667,53.841648,4,...,3243,02 SLB,8,20.100106,0.943279,0.186069,-7.158352,0.000000,0,-1.140106
2,SE-064c0cec-1189,a624fb9a,2022-05-30,22.64,18.96,3.68,0,10.466667,52.935959,5,...,3244,02 SLB,8,24.743239,0.914997,0.148727,-8.064041,0.000000,0,-2.103239
3,SE-064c0cec-1189,a624fb9a,2022-05-31,26.49,22.64,3.85,0,11.183333,52.872112,6,...,3245,02 SLB,8,28.284407,0.936558,0.136117,-8.127888,0.000000,0,-1.794407
4,SE-064c0cec-1189,a624fb9a,2022-06-01,33.61,26.49,7.12,0,12.704167,56.056547,7,...,3246,02 SLB,8,30.980818,1.084865,0.22982,-4.943453,0.000000,0,2.629182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583674,SE-fcdf259d-0044-0,f454e660,2023-06-08,15.14,8.97,6.17,0,15.570833,59.383267,352,...,4155,41 Fjällko,10,11.914894,1.270678,0.517839,-1.616733,0.000000,0,3.225106
583675,SE-fcdf259d-0044-0,f454e660,2023-06-09,7.47,15.14,-7.67,0,13.254167,54.534255,353,...,4156,41 Fjällko,10,10.937277,0.682985,-0.701271,-6.465745,0.000000,0,-3.467277
583676,SE-fcdf259d-0044-0,f454e660,2023-06-10,14.73,7.47,7.26,0,13.258333,54.082367,354,...,4157,41 Fjällko,10,12.646376,1.164761,0.574077,-6.917633,0.000000,0,2.083624
583677,SE-fcdf259d-0044-0,f454e660,2023-06-12,12.27,14.73,-2.46,0,15.820833,62.015093,356,...,4159,41 Fjällko,10,11.04863,1.110545,-0.222652,1.015093,1.015093,0,1.22137


In [23]:
# Reorder columns
new_order = [
    "Date", "FarmName_Pseudo", "SE_Number", "Age", "BreedName", "LactationNumber", "DaysInMilk",'YearSeason', "DailyYield", "PreviousDailyYield", 
    "DailyYieldChange", "ExpectedYield", "NormalizedDailyYield", 
    "NormalizedDailyYieldChange", "Residuals", "HeatStress", "Temp15Threshold", "HW", 
    "cum_HW", "MeanTemperature", "MeanTHI_adj", "HeatLoad", "CumulativeHeatLoad"
]
data_cleaned = data_cleaned[new_order]
data_cleaned.head()

,Date,FarmName_Pseudo,SE_Number,Age,BreedName,LactationNumber,DaysInMilk,YearSeason,DailyYield,PreviousDailyYield,...,NormalizedDailyYieldChange,Residuals,HeatStress,Temp15Threshold,HW,cum_HW,MeanTemperature,MeanTHI_adj,HeatLoad,CumulativeHeatLoad
0,2022-05-28,a624fb9a,SE-064c0cec-1189,3242,02 SLB,8,3,2022-2,15.22,NaN,...,<NA>,1.200512,0,0,0,0,9.912500,50.478673,-10.521327,0.0
1,2022-05-29,a624fb9a,SE-064c0cec-1189,3243,02 SLB,8,4,2022-2,18.96,15.22,...,0.186069,-1.140106,0,0,0,0,10.066667,53.841648,-7.158352,0.0
2,2022-05-30,a624fb9a,SE-064c0cec-1189,3244,02 SLB,8,5,2022-2,22.64,18.96,...,0.148727,-2.103239,0,1,0,0,10.466667,52.935959,-8.064041,0.0
3,2022-05-31,a624fb9a,SE-064c0cec-1189,3245,02 SLB,8,6,2022-2,26.49,22.64,...,0.136117,-1.794407,0,0,0,0,11.183333,52.872112,-8.127888,0.0
4,2022-06-01,a624fb9a,SE-064c0cec-1189,3246,02 SLB,8,7,2022-3,33.61,26.49,...,0.22982,2.629182,0,1,0,0,12.704167,56.056547,-4.943453,0.0


In [24]:
# Check if NormalizedDailyYield is centered around 1 for each unique farm
print("Mean of NormalizedDailyYield:", data_cleaned.groupby('FarmName_Pseudo')['NormalizedDailyYield'].mean())
print("Standard Deviation of NormalizedDailyYield:", data_cleaned.groupby('FarmName_Pseudo')['NormalizedDailyYield'].std())

Mean of NormalizedDailyYield: FarmName_Pseudo
5c06d92d    0.999634
752efd72    0.999387
a624fb9a    0.997566
f454e660     0.99873
Name: NormalizedDailyYield, dtype: Float64
Standard Deviation of NormalizedDailyYield: FarmName_Pseudo
5c06d92d    0.121557
752efd72    0.096416
a624fb9a    0.176894
f454e660    0.231439
Name: NormalizedDailyYield, dtype: Float64


In [25]:
# Count the number of HeatStress occurrences in each farm
heat_stress_counts = data_cleaned.groupby('FarmName_Pseudo')['HeatStress'].sum()
heat_stress_counts

FarmName_Pseudo
5c06d92d    61521
752efd72    45494
a624fb9a    18963
f454e660    20980
Name: HeatStress, dtype: int64